In [1]:
# Model Selection using Keras Tuner

In [2]:
import os
import sys

os.environ["CUDA_VISIBLE_DEVICES"]="0" # Nvidia Quadro GV100
#os.environ["CUDA_VISIBLE_DEVICES"]="1" # Nvidia Quadro M2000

#Adding Path to various Modules
sys.path.append("../core")
sys.path.append("../visualization")
sys.path.append("../utilities")
sys.path.append("../datasets")
sys.path.append("../trained_models")
sys.path.append("../config")

In [3]:
#Importing Required Modules
import pathlib
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import backend as K
K.clear_session()

#Importing Config files
import assembly_config as config
import model_config as cftrain
import hybrid_utils as hy_util
#Importing required modules from the package
from measurement_system import HexagonWlsScanner
from assembly_system import VRMSimulationModel
from wls400a_system import GetInferenceData
from data_import import GetTrainData
from encode_decode_model import Encode_Decode_Model
from training_viz import TrainViz
from metrics_eval import MetricsEval
from keras_lr_multiplier import LRMultiplier

Using TensorFlow backend.


In [4]:
print('Parsing from Assembly Config File....')

data_type=config.assembly_system['data_type']
application=config.assembly_system['application']
part_type=config.assembly_system['part_type']
part_name=config.assembly_system['part_name']
data_format=config.assembly_system['data_format']
assembly_type=config.assembly_system['assembly_type']
assembly_kccs=config.assembly_system['assembly_kccs']
assembly_kpis=config.assembly_system['assembly_kpis']
voxel_dim=config.assembly_system['voxel_dim']
point_dim=config.assembly_system['point_dim']
voxel_channels=config.assembly_system['voxel_channels']
noise_type=config.assembly_system['noise_type']
mapping_index=config.assembly_system['mapping_index']

system_noise=config.assembly_system['system_noise']
aritifical_noise=config.assembly_system['aritifical_noise']
data_folder=config.assembly_system['data_folder']
kcc_folder=config.assembly_system['kcc_folder']
kcc_files=config.assembly_system['kcc_files']
test_kcc_files=config.assembly_system['test_kcc_files']

Parsing from Assembly Config File....


In [5]:
#added for hybrid model
categorical_kccs=config.assembly_system['categorical_kccs']

print('Parsing from Training Config File')

model_type=cftrain.model_parameters['model_type']
output_type=cftrain.model_parameters['output_type']
batch_size=cftrain.model_parameters['batch_size']
epocs=cftrain.model_parameters['epocs']
split_ratio=cftrain.model_parameters['split_ratio']
optimizer=cftrain.model_parameters['optimizer']
loss_func=cftrain.model_parameters['loss_func']
regularizer_coeff=cftrain.model_parameters['regularizer_coeff']
activate_tensorboard=cftrain.model_parameters['activate_tensorboard']

Parsing from Training Config File


In [6]:
print('Creating file Structure....')

folder_name=part_type
train_path='../trained_models/'+part_type
pathlib.Path(train_path).mkdir(parents=True, exist_ok=True) 

train_path=train_path+'/model_selection'
pathlib.Path(train_path).mkdir(parents=True, exist_ok=True) 

model_path=train_path+'/models'
pathlib.Path(model_path).mkdir(parents=True, exist_ok=True)

logs_path=train_path+'/selection_logs'
pathlib.Path(logs_path).mkdir(parents=True, exist_ok=True)

plots_path=train_path+'/plots'
pathlib.Path(plots_path).mkdir(parents=True, exist_ok=True)

tuned_path=train_path+'/tuned_model'
pathlib.Path(tuned_path).mkdir(parents=True, exist_ok=True)

Creating file Structure....


In [7]:
#Objects of Measurement System, Assembly System, Get Inference Data
print('Initializing the Assembly System and Measurement System....')
measurement_system=HexagonWlsScanner(data_type,application,system_noise,part_type,data_format)
vrm_system=VRMSimulationModel(assembly_type,assembly_kccs,assembly_kpis,part_name,part_type,voxel_dim,voxel_channels,point_dim,aritifical_noise)
get_data=GetTrainData()


kcc_sublist=cftrain.encode_decode_params['kcc_sublist']
output_heads=cftrain.encode_decode_params['output_heads']
encode_decode_multi_output_construct=config.encode_decode_multi_output_construct

if(output_heads==len(encode_decode_multi_output_construct)):
	print("Valid Output Stages and heads")
else:
	print("Inconsistent model setting")

print("KCC sub-listing: ",kcc_sublist)

#Check for KCC sub-listing
if(kcc_sublist!=0):
	output_dimension=len(kcc_sublist)
else:
	output_dimension=assembly_kccs

print("Process Parameter Dimension: ",output_dimension)

input_size=(voxel_dim,voxel_dim,voxel_dim,voxel_channels)

model_depth=cftrain.encode_decode_params['model_depth']
inital_filter_dim=cftrain.encode_decode_params['inital_filter_dim']


Initializing the Assembly System and Measurement System....
Valid Output Stages and heads
KCC sub-listing:  0
Process Parameter Dimension:  157


In [8]:
#importing file names for model input
input_file_names_x=config.encode_decode_construct['input_data_files_x']
input_file_names_y=config.encode_decode_construct['input_data_files_y']
input_file_names_z=config.encode_decode_construct['input_data_files_z']

input_dataset=[]
input_dataset.append(get_data.data_import(input_file_names_x,data_folder))
input_dataset.append(get_data.data_import(input_file_names_y,data_folder))
input_dataset.append(get_data.data_import(input_file_names_z,data_folder))

kcc_dataset=get_data.data_import(kcc_files,kcc_folder)

if(kcc_sublist!=0):
	print("Sub-setting Process Parameters: ",kcc_sublist)
	kcc_dataset=kcc_dataset.iloc[:,kcc_sublist]
	test_kcc_dataset=test_kcc_dataset[:,kcc_sublist]
else:
	print("Using all Process Parameter")

Using all Process Parameter


In [9]:
#Pre-processing to point cloud data
point_index=get_data.load_mapping_index(mapping_index)
input_conv_data, kcc_subset_dump,kpi_subset_dump=get_data.data_convert_voxel_mc(vrm_system,input_dataset,point_index,kcc_dataset)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:40<00:00, 12.40it/s]

Number of not convergent solutions:  0


In [10]:
kcc_regression,kcc_classification=hy_util.split_kcc(kcc_subset_dump)
Y_out_list=[]
Y_out_list.append(kcc_regression)
Y_out_list.append(kcc_classification)

Splitting Contionous and Categorical KCCs
Valid Split


In [11]:
y_shape_error_list=[]

for encode_decode_construct in encode_decode_multi_output_construct:
	#importing file names for model output
	print("Importing output data for stage: ",encode_decode_construct)

	output_file_names_x=encode_decode_construct['output_data_files_x']
	output_file_names_y=encode_decode_construct['output_data_files_y']
	output_file_names_z=encode_decode_construct['output_data_files_z']

	output_dataset=[]
	output_dataset.append(get_data.data_import(output_file_names_x,data_folder))
	output_dataset.append(get_data.data_import(output_file_names_y,data_folder))
	output_dataset.append(get_data.data_import(output_file_names_z,data_folder))

	output_conv_data, kcc_subset_dump,kpi_subset_dump=get_data.data_convert_voxel_mc(vrm_system,output_dataset,point_index,kcc_dataset)

	y_shape_error_list.append(output_conv_data)

shape_error=np.concatenate(y_shape_error_list, axis=4)
Y_out_list.append(shape_error)

Importing output data for stage:  {'stage_id': 2, 'output_data_files_x': ['DX_crossmember_test1_3.csv'], 'output_data_files_y': ['DY_crossmember_test1_3.csv'], 'output_data_files_z': ['DZ_crossmember_test1_3.csv'], 'output_test_data_files_x': ['DX_crossmember_test1_3.csv'], 'output_test_data_files_y': ['DY_crossmember_test1_3.csv'], 'output_test_data_files_z': ['DZ_crossmember_test1_3.csv']}


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:40<00:00, 12.36it/s]


Number of not convergent solutions:  0
Importing output data for stage:  {'stage_id': 6, 'output_data_files_x': ['DX_crossmember_test1_7.csv'], 'output_data_files_y': ['DY_crossmember_test1_7.csv'], 'output_data_files_z': ['DZ_crossmember_test1_7.csv'], 'output_test_data_files_x': ['DX_crossmember_test1_7.csv'], 'output_test_data_files_y': ['DY_crossmember_test1_7.csv'], 'output_test_data_files_z': ['DZ_crossmember_test1_7.csv']}


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:40<00:00, 12.42it/s]


Number of not convergent solutions:  0


In [23]:
# Build Model using Keras Tuner

#Get Base Model
#Reloading to change cache
import importlib
from model_base_arch import *
#importlib.reload(model_base_arch)

base_model_obj4=BaseModelArch(output_dimension)


In [24]:
#Declare Model Hyperparameters to Tune
import kerastuner as kt
from kerastuner.tuners import RandomSearch

#HyperModel Defination
def model_builder(hp):
    
    base_model=base_model_obj4.base_model_func(hp,inital_filter_dim,model_depth,categorical_kccs,voxel_dim=64,deviation_channels=3,output_heads=2)
    
    return base_model
    
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)


In [31]:
#Define Objective and direction to converge In
from tensorflow.keras import backend as K
K.clear_session()
os.environ["CUDA_VISIBLE_DEVICES"]="0" 
obj=kt.Objective("regression_outputs", direction="min")

tuner = kt.Hyperband(model_builder,
                     objective = 'val_loss', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'OSER_A',
                     project_name = 'OSER_Arch_Opt')

tuner.search(input_conv_data, Y_out_list, validation_split=0.2, batch_size=16, epochs = 10)

3D CNN model successfully compiled
3D CNN model successfully compiled
Train on 400 samples, validate on 100 samples
Epoch 1/2
400/400 [==============================] - ETA: 2:45 - loss: 3.0347 - regression_outputs_loss: 0.8334 - classification_outputs_loss: 0.6799 - shape_error_outputs_loss: 0.0080 - regression_outputs_mean_absolute_error: 0.6580 - classification_outputs_categorical_accuracy: 0.0000e+00 - shape_error_outputs_mean_absolute_error: 0.022 - ETA: 1:34 - loss: 3.4639 - regression_outputs_loss: 1.1684 - classification_outputs_loss: 0.5117 - shape_error_outputs_loss: 0.1037 - regression_outputs_mean_absolute_error: 0.8137 - classification_outputs_categorical_accuracy: 0.0000e+00 - shape_error_outputs_mean_absolute_error: 0.141 - ETA: 1:09 - loss: 3.2903 - regression_outputs_loss: 1.0614 - classification_outputs_loss: 0.5485 - shape_error_outputs_loss: 0.0705 - regression_outputs_mean_absolute_error: 0.7678 - classification_outputs_categorical_accuracy: 0.0000e+00 - shape_erro

400/400 [==============================] - ETA: 30s - loss: 2.2129 - regression_outputs_loss: 0.8858 - classification_outputs_loss: 0.2187 - shape_error_outputs_loss: 0.0040 - regression_outputs_mean_absolute_error: 0.7192 - classification_outputs_categorical_accuracy: 0.0000e+00 - shape_error_outputs_mean_absolute_error: 0.00 - ETA: 29s - loss: 2.1313 - regression_outputs_loss: 0.8790 - classification_outputs_loss: 0.1846 - shape_error_outputs_loss: 0.0042 - regression_outputs_mean_absolute_error: 0.7194 - classification_outputs_categorical_accuracy: 0.0000e+00 - shape_error_outputs_mean_absolute_error: 0.00 - ETA: 28s - loss: 2.2269 - regression_outputs_loss: 0.8902 - classification_outputs_loss: 0.2212 - shape_error_outputs_loss: 0.0040 - regression_outputs_mean_absolute_error: 0.7247 - classification_outputs_categorical_accuracy: 0.0000e+00 - shape_error_outputs_mean_absolute_error: 0.00 - ETA: 27s - loss: 2.1145 - regression_outputs_loss: 0.8945 - classification_outputs_loss: 0.16

3D CNN model successfully compiled
Train on 400 samples, validate on 100 samples
Epoch 1/2


384/400 [===========================>..] - ETA: 2:37 - loss: 3.2199 - regression_outputs_loss: 0.9158 - classification_outputs_loss: 0.6914 - shape_error_outputs_loss: 0.0054 - regression_outputs_mean_absolute_error: 0.7192 - classification_outputs_categorical_accuracy: 0.0625 - shape_error_outputs_mean_absolute_error: 0.014 - ETA: 1:28 - loss: 8509732915335135232.0000 - regression_outputs_loss: 82731016192.0000 - classification_outputs_loss: 0.6246 - shape_error_outputs_loss: 8509732915335135232.0000 - regression_outputs_mean_absolute_error: 170305.6094 - classification_outputs_categorical_accuracy: 0.0312 - shape_error_outputs_mean_absolute_error: 1504327296.000 - ETA: 1:04 - loss: 5673155276890090496.0000 - regression_outputs_loss: 55154012160.0000 - classification_outputs_loss: 0.4825 - shape_error_outputs_loss: 5673155093638152192.0000 - regression_outputs_mean_absolute_error: 113537.6328 - classification_outputs_categorical_accuracy: 0.0208 - shape_error_outputs_mean_absolute_err

400/400 [==============================] - 39s 98ms/sample - loss: 680778633226810880.0000 - regression_outputs_loss: 6618481152.0000 - classification_outputs_loss: 0.6224 - shape_error_outputs_loss: 680778635975589888.0000 - regression_outputs_mean_absolute_error: 13625.1240 - classification_outputs_categorical_accuracy: 0.0025 - shape_error_outputs_mean_absolute_error: 120346184.0000 - val_loss: 2.9873 - val_regression_outputs_loss: 0.8878 - val_classification_outputs_loss: 0.6061 - val_shape_error_outputs_loss: 0.0066 - val_regression_outputs_mean_absolute_error: 0.6986 - val_classification_outputs_categorical_accuracy: 0.0000e+00 - val_shape_error_outputs_mean_absolute_error: 0.0481
Epoch 2/2
400/400 [==============================] - ETA: 26s - loss: 3.0482 - regression_outputs_loss: 0.9199 - classification_outputs_loss: 0.6008 - shape_error_outputs_loss: 0.0069 - regression_outputs_mean_absolute_error: 0.7282 - classification_outputs_categorical_accuracy: 0.0000e+00 - shape_error

...

400/400 [==============================] - ETA: 24s - loss: 1.9279 - regression_outputs_loss: 0.9393 - classification_outputs_loss: 0.0226 - shape_error_outputs_loss: 0.0041 - regression_outputs_mean_absolute_error: 0.7460 - classification_outputs_categorical_accuracy: 0.0000e+00 - shape_error_outputs_mean_absolute_error: 0.01 - ETA: 23s - loss: 1.9429 - regression_outputs_loss: 0.9000 - classification_outputs_loss: 0.0694 - shape_error_outputs_loss: 0.0042 - regression_outputs_mean_absolute_error: 0.7242 - classification_outputs_categorical_accuracy: 0.0000e+00 - shape_error_outputs_mean_absolute_error: 0.01 - ETA: 22s - loss: 1.9430 - regression_outputs_loss: 0.9058 - classification_outputs_loss: 0.0637 - shape_error_outputs_loss: 0.0040 - regression_outputs_mean_absolute_error: 0.7279 - classification_outputs_categorical_accuracy: 0.0000e+00 - shape_error_outputs_mean_absolute_error: 0.01 - ETA: 21s - loss: 2.0973 - regression_outputs_loss: 0.9183 - classification_outputs_loss: 0.12

400/400 [==============================] - ETA: 24s - loss: 1.7370 - regression_outputs_loss: 0.8510 - classification_outputs_loss: 0.0150 - shape_error_outputs_loss: 0.0049 - regression_outputs_mean_absolute_error: 0.7057 - classification_outputs_categorical_accuracy: 0.0000e+00 - shape_error_outputs_mean_absolute_error: 0.01 - ETA: 23s - loss: 1.7115 - regression_outputs_loss: 0.8558 - classification_outputs_loss: -0.0021 - shape_error_outputs_loss: 0.0042 - regression_outputs_mean_absolute_error: 0.7066 - classification_outputs_categorical_accuracy: 0.0000e+00 - shape_error_outputs_mean_absolute_error: 0.014 - ETA: 22s - loss: 1.8284 - regression_outputs_loss: 0.8623 - classification_outputs_loss: 0.0499 - shape_error_outputs_loss: 0.0041 - regression_outputs_mean_absolute_error: 0.7072 - classification_outputs_categorical_accuracy: 0.0000e+00 - shape_error_outputs_mean_absolute_error: 0.014 - ETA: 21s - loss: 1.7038 - regression_outputs_loss: 0.8610 - classification_outputs_loss: -

400/400 [==============================] - ETA: 25s - loss: 1.8982 - regression_outputs_loss: 0.8535 - classification_outputs_loss: 0.0939 - shape_error_outputs_loss: 0.0033 - regression_outputs_mean_absolute_error: 0.7000 - classification_outputs_categorical_accuracy: 0.0000e+00 - shape_error_outputs_mean_absolute_error: 0.01 - ETA: 24s - loss: 2.0853 - regression_outputs_loss: 0.8812 - classification_outputs_loss: 0.1595 - shape_error_outputs_loss: 0.0039 - regression_outputs_mean_absolute_error: 0.7191 - classification_outputs_categorical_accuracy: 0.0000e+00 - shape_error_outputs_mean_absolute_error: 0.01 - ETA: 23s - loss: 2.0166 - regression_outputs_loss: 0.8842 - classification_outputs_loss: 0.1222 - shape_error_outputs_loss: 0.0037 - regression_outputs_mean_absolute_error: 0.7194 - classification_outputs_categorical_accuracy: 0.0000e+00 - shape_error_outputs_mean_absolute_error: 0.01 - ETA: 21s - loss: 1.7456 - regression_outputs_loss: 0.8742 - classification_outputs_loss: -0.0

400/400 [==============================] - ETA: 24s - loss: 1.2727 - regression_outputs_loss: 0.8596 - classification_outputs_loss: -0.2250 - shape_error_outputs_loss: 0.0035 - regression_outputs_mean_absolute_error: 0.7054 - classification_outputs_categorical_accuracy: 0.0000e+00 - shape_error_outputs_mean_absolute_error: 0.010 - ETA: 23s - loss: 1.6961 - regression_outputs_loss: 0.8402 - classification_outputs_loss: 0.0059 - shape_error_outputs_loss: 0.0040 - regression_outputs_mean_absolute_error: 0.6960 - classification_outputs_categorical_accuracy: 0.0000e+00 - shape_error_outputs_mean_absolute_error: 0.010 - ETA: 22s - loss: 1.8026 - regression_outputs_loss: 0.8400 - classification_outputs_loss: 0.0592 - shape_error_outputs_loss: 0.0042 - regression_outputs_mean_absolute_error: 0.6981 - classification_outputs_categorical_accuracy: 0.0000e+00 - shape_error_outputs_mean_absolute_error: 0.01 - ETA: 21s - loss: 1.5548 - regression_outputs_loss: 0.8308 - classification_outputs_loss: -

INFO:tensorflow:Oracle triggered exit
